In [1]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
options.add_argument("User-Agent:Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36")
options.add_argument("lang=ko_KR")

chromedriver = '/usr/local/chromedriver/chromedriver'
driver = webdriver.Chrome(service=Service(chromedriver), options=options)
# 크롤링할 사이트
driver.get('https://place-site.yanolja.com/places/1010682', )

# 첫 접속 시 나오는 튜토리얼 페이지 스킵
for i in range(3):
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,"div.right.css-oq3qu5"))).click()

# 리뷰 페이지로 이동
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,"#__next > div > div > main > article > div:nth-child(2) > div > div.css-ca9ity > div > button:nth-child(5) > a"))).click()

"""
# 리뷰가 가장 적은 곳으로 이동 (test용)
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,"#__next > div > div > main > article > div:nth-child(2) > div > div.css-17c0wg8 > div > div:nth-child(2) > div > div.css-11ega0p > span"))).click()
time.sleep(5)
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,"#BOTTOM_SHEET > div.css-gqqlqe > li:nth-child(5)"))).click()
"""

# 스크롤링하면서 리뷰 더 보기
SCROLL_PAUSE_TIME = 1.5
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    # 스크롤링 4번
    for i in range(4):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)    
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    names_dates = driver.find_elements(By.CSS_SELECTOR, "div.css-1fd7prz")
    reviews = driver.find_elements(By.CSS_SELECTOR, "div.css-1wnvjz2")
    
    # 스크롤링 종료 조건
    if new_height == last_height:
        break
    last_height = new_height

# 크롤링 데이터 데이터프레임화
result_dict = []
for i in range(len(names_dates)):
    result_dict.append({
        'NAME,DATE' : names_dates[i].text,
        'REVIEW' : reviews[i].text
    })
result_df = pd.DataFrame(result_dict)

driver.quit()

In [3]:
# 'NAME,DATE' 칼럼을 각각 'NAME', 'DATE'로 변경
result_df['NAME,DATE'] = result_df['NAME,DATE'].astype('str')
col_name_list = result_df['NAME,DATE'].str.split('\n')
result_df['NAME'] = col_name_list.str.get(0)
result_df['DATE'] = col_name_list.str.get(1)
result_df = result_df[['NAME', 'DATE', 'REVIEW']]

# 현 디렉터리에 .xlsx로 저장
result_df.to_excel('result.xlsx')